In [13]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Input, Reshape, Layer
from tensorflow.keras.optimizers import SGD, RMSprop, Adam
from matplotlib import pyplot as plt
import numpy as np
import matplotlib.pyplot as plt
from time import time
import random

In [62]:
loss_tracker = keras.metrics.Mean(name="loss")
from tensorflow.python.ops.numpy_ops import np_config
np_config.enable_numpy_behavior()
class Replicar(Sequential):
    @property
    def metrics(self):
        return [loss_tracker] 

    def train_step(self, data):
        def funcA(x):
            return tf.math.sin(np.pi*x)*3

        def funcB(x):
            return 1 + 2*x + 4*x**3

        interval = [-1, 1]
        batch_size = 100 #Calibra la resolucion
        x = tf.random.uniform((batch_size, 1), minval=-1, maxval=1)
        inter = np.linspace(interval, 100)
        vals = funcA(inter).astype(np.float32)

        with tf.GradientTape() as tape:
            y_pred = self(x, training=True)
            loss = tf.math.reduce_mean(tf.math.square(y_pred-vals))

        grads = tape.gradient(loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        loss_tracker.update_state(loss)

        return {"loss": loss_tracker.result()}

In [35]:
def funcA(x):
    return tf.math.sin(np.pi*x)*3

def funcB(x):
    return 1 + 2*x + 4*x**3

def noise(f, x):
    vals = f(x)
    for i in vals:
        vals += random.uniform(-0.3, 0.3)
    return vals

interval = [-1, 1]

In [63]:
model = Replicar([
    Dense(600,activation='tanh', input_shape=(1,)),
    Dense(200,activation='tanh'),
    Dense(200,activation='relu'),
    Dense(1),
])

model.summary()

Model: "replicar_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_28 (Dense)            (None, 600)               1200      
                                                                 
 dense_29 (Dense)            (None, 200)               120200    
                                                                 
 dense_30 (Dense)            (None, 200)               40200     
                                                                 
 dense_31 (Dense)            (None, 1)                 201       
                                                                 
Total params: 161,801
Trainable params: 161,801
Non-trainable params: 0
_________________________________________________________________


In [66]:
dummy_data = noise(funcA, np.linspace(-1, 1, 100))  # Replace with your data
model.compile(optimizer=Adam(learning_rate=0.0001), metrics=['loss'])
history = model.fit(dummy_data, epochs=1000)

Epoch 1/1000


ValueError: in user code:

    File "C:\Users\USER\anaconda3\envs\tf\lib\site-packages\keras\engine\training.py", line 1160, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\USER\anaconda3\envs\tf\lib\site-packages\keras\engine\training.py", line 1146, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\USER\anaconda3\envs\tf\lib\site-packages\keras\engine\training.py", line 1135, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\USER\AppData\Local\Temp\ipykernel_12552\1954137025.py", line 24, in train_step
        loss = tf.math.reduce_mean(tf.math.square(y_pred-vals))

    ValueError: Dimensions must be equal, but are 100 and 50 for '{{node sub}} = Sub[T=DT_FLOAT](replicar_8/dense_31/BiasAdd, Cast)' with input shapes: [100,1], [50,2].


In [65]:
dummy_data

<tf.Tensor: shape=(100,), dtype=float64, numpy=
array([ 1.47082722,  1.28055546,  1.09104986,  0.90307348,  0.71738326,
        0.53472688,  0.35583985,  0.18144248,  0.01223701, -0.15109523,
       -0.30789657, -0.45753561, -0.59940982, -0.73294791, -0.85761217,
       -0.97290064, -1.07834907, -1.17353287, -1.25806877, -1.33161636,
       -1.39387951, -1.44460749, -1.48359604, -1.51068818, -1.5257748 ,
       -1.52879517, -1.51973711, -1.49863711, -1.46558012, -1.42069926,
       -1.36417524, -1.29623566, -1.2171541 , -1.12724899, -1.02688235,
       -0.9164583 , -0.79642151, -0.6672553 , -0.52947978, -0.38364974,
       -0.23035237, -0.07020496,  0.09614765,  0.26803561,  0.44476679,
        0.62562955,  0.80989562,  0.99682303,  1.18565909,  1.37564342,
        1.56601102,  1.75599535,  1.94483141,  2.13175882,  2.31602489,
        2.49688765,  2.67361882,  2.84550678,  3.01185939,  3.17200681,
        3.32530418,  3.47113422,  3.60890973,  3.73807594,  3.85811274,
        3.968536